In [1]:
import pandas as pd 
import numpy as np 
import os
import math 
import matplotlib.pyplot as plt
import seaborn as sns
import time
sns.set()

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold



In [2]:
path = "."
fname = os.path.join(path, "edu_imputed.csv")
df = pd.read_csv(fname)
df = df.drop(columns = 'Unnamed: 0')
df.head()

C:\Users\adcm172\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,title,totalyearlycompensation,yearsofexperience,Education,tag,gender,title_Business Analyst,title_Data Scientist,title_Hardware Engineer,title_Human Resources,...,tag_iOS,gender_Female,gender_Male,gender_Other,tcomp_scaled,yexp_scaled,edu_encoded,row_type,edu_imputed,final_edu_val
0,Software Engineer,78000,4.0,Bachelor's Degree,Full Stack,Male,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,-0.994331,0.095238,3.0,train,NaN,3.0
1,Software Engineer,42000,10.0,Master's Degree,Web Development (Front-End),Male,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,-1.250724,0.238095,4.0,train,NaN,4.0
2,Software Engineering Manager,458000,15.0,Master's Degree,Site Reliability (SRE),Female,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,1.712039,0.357143,4.0,train,NaN,4.0
3,Software Engineer,163000,10.0,Master's Degree,Distributed Systems (Back-End),Male,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,-0.388958,0.238095,4.0,train,NaN,4.0
4,Software Engineer,120000,6.0,Master's Degree,API Development (Back-End),Female,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,-0.695206,0.142857,4.0,train,NaN,4.0


In [5]:
#df = df[df.totalyearlycompensation < 1000000]
#df.count().max()

53753

In [6]:
#features_500dp = df[features].sum() < 500
#features_ = features_500dp[features_500dp == True].index
#features_

In [7]:
#df = df[df['title'].isin(['Human Resources', 'Management Consultant', 'Marketing', 'Mechanical Engineer', 'Recruiter', 'Sales'])==False]
#df.count().max()

52027

In [8]:
#df = df[df['gender'].isin(['Other'])==False]
#df.count().max()

51546

In [3]:
df.columns

Index(['title', 'totalyearlycompensation', 'yearsofexperience', 'Education',
       'tag', 'gender', 'title_Business Analyst', 'title_Data Scientist',
       'title_Hardware Engineer', 'title_Human Resources',
       'title_Management Consultant', 'title_Marketing',
       'title_Mechanical Engineer', 'title_Product Designer',
       'title_Product Manager', 'title_Recruiter', 'title_Sales',
       'title_Software Engineer', 'title_Software Engineering Manager',
       'title_Solution Architect', 'title_Technical Program Manager',
       'tag_API Development (Back-End)', 'tag_Android', 'tag_Data',
       'tag_DevOps', 'tag_Distributed Systems (Back-End)', 'tag_Full Stack',
       'tag_ML / AI', 'tag_Mobile (iOS + Android)', 'tag_Networking',
       'tag_Other', 'tag_Production', 'tag_Security',
       'tag_Site Reliability (SRE)', 'tag_Testing (SDET)',
       'tag_Web Development (Front-End)', 'tag_iOS', 'gender_Female',
       'gender_Male', 'gender_Other', 'tcomp_scaled', 'yexp_scale

In [4]:
features = ['title_Business Analyst', 'title_Data Scientist',
       'title_Hardware Engineer', 'title_Human Resources',
       'title_Management Consultant', 'title_Marketing',
       'title_Mechanical Engineer', 'title_Product Designer',
       'title_Product Manager', 'title_Recruiter', 'title_Sales',
       'title_Software Engineer', 'title_Software Engineering Manager',
       'title_Solution Architect', 'title_Technical Program Manager',
       'tag_API Development (Back-End)', 'tag_Android', 'tag_Data',
       'tag_DevOps', 'tag_Distributed Systems (Back-End)', 'tag_Full Stack',
       'tag_ML / AI', 'tag_Mobile (iOS + Android)', 'tag_Networking',
       'tag_Other', 'tag_Production', 'tag_Security',
       'tag_Site Reliability (SRE)', 'tag_Testing (SDET)',
       'tag_Web Development (Front-End)', 'tag_iOS', 'gender_Female',
       'gender_Male', 'gender_Other', 'yexp_scaled', 'final_edu_val']
#features = list(set(features) - set(features_))
target = 'totalyearlycompensation'

y = df[target]
X = df[features].values

In [5]:
#can't use PCA for non-continuous variable 

In [6]:
np.random.seed(1023)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [7]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(43088, 36) (10772, 36) (43088,) (10772,)


In [8]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR


In [9]:
classifiers = [
    LinearRegression(),
    SGDRegressor(max_iter=1000),
    RandomForestRegressor(),
    GradientBoostingRegressor(random_state=0)
    #,SVR()
    
]
classifier_names = [
    'Linear Regression Regression' ,
    'SDG Regressor',
    'Random Forest Regressor', 
    'Gradient Boosted Regressor' 
    #,'Support Vector Machine'
    

]

In [10]:
for clf, clf_name in zip(classifiers, classifier_names):
    time_start = time.time()
    cv_scores = cross_val_score(clf, X, np.ravel(y), cv=5)
    print(clf_name, ' mean accuracy: ',
          round(cv_scores.mean()*100, 3), 
          '% std: ', round(cv_scores.var()*100, 3),'%',
          'training time: ', round(time.time() - time_start, 2), 's'
         )

Linear Regression Regression  mean accuracy:  26.585 % std:  0.33 % training time:  0.38 s
SDG Regressor  mean accuracy:  26.283 % std:  0.302 % training time:  1.08 s
Random Forest Regressor  mean accuracy:  19.372 % std:  2.231 % training time:  40.43 s
Gradient Boosted Regressor  mean accuracy:  28.09 % std:  0.895 % training time:  19.91 s


In [11]:

#use hyper parameter tuning for Gradient Boosted Regressor
time_start = time.time()
model = GradientBoostingRegressor()

#https://www.analyticsvidhya.com/blog/2016/02/complete-guide-parameter-tuning-gradient-boosting-gbm-python/

grid={'min_samples_split': [10,20,30], #Defines the minimum number of samples which are required in a node to be considered for splitting
      'min_samples_leaf':[100,300,500], #Defines the minimum samples (or observations) required in a terminal node or leaf
      'max_depth':[8,10], #The maximum depth of a tree
      'learning_rate':[.1], # controls the magnitude of this change in the estimates
      'subsample':[.8],
      'random_state':[23]}
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=3, random_state=1)
grid_search=GridSearchCV(estimator=model,param_grid=grid,n_jobs=1,cv=cv)
grid_result = grid_search.fit(X, y)


# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
print('time taken: ', time.time()-time_start, 's')


C:\Users\adcm172\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\adcm172\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"
C:\Users\adcm172\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(("The least populated class in y has only %d"


Best: 0.314537 using {'learning_rate': 0.1, 'max_depth': 8, 'min_samples_leaf': 100, 'min_samples_split': 10, 'random_state': 23, 'subsample': 0.8}
0.314537 (0.013259) with: {'learning_rate': 0.1, 'max_depth': 8, 'min_samples_leaf': 100, 'min_samples_split': 10, 'random_state': 23, 'subsample': 0.8}
0.314537 (0.013259) with: {'learning_rate': 0.1, 'max_depth': 8, 'min_samples_leaf': 100, 'min_samples_split': 20, 'random_state': 23, 'subsample': 0.8}
0.314537 (0.013259) with: {'learning_rate': 0.1, 'max_depth': 8, 'min_samples_leaf': 100, 'min_samples_split': 30, 'random_state': 23, 'subsample': 0.8}
0.312220 (0.013458) with: {'learning_rate': 0.1, 'max_depth': 8, 'min_samples_leaf': 300, 'min_samples_split': 10, 'random_state': 23, 'subsample': 0.8}
0.312220 (0.013458) with: {'learning_rate': 0.1, 'max_depth': 8, 'min_samples_leaf': 300, 'min_samples_split': 20, 'random_state': 23, 'subsample': 0.8}
0.312220 (0.013458) with: {'learning_rate': 0.1, 'max_depth': 8, 'min_samples_leaf': 30

Best: 0.314537 using {'learning_rate': 0.1, 'max_depth': 8, 'min_samples_leaf': 100, 'min_samples_split': 10, 'random_state': 23, 'subsample': 0.8}

In [17]:
## Gradient Boosted Regressor

In [13]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import r2_score


def display_results(y_pred, y_test):
    print("Predicted 1-5:", np.array([round(y_pred[x][0]) for x in range(0,len(y_pred))][0:10]))
    print("Actual 1-5:", y_test[0:10].values)
    print("Difference: ", np.array([round(y_pred[x][0]) for x in range(0,len(y_pred))][0:10]) - np.array(y_test[0:10].values))
    #count number of correct predictions
    print('RMSE: ', np.sqrt(mean_absolute_error(y_test, y_pred)))
    print('R-squared: ', r2_score(y_test, y_pred) )
    #y_diff = y_pred - y_test 
    #temp = [y_diff[x][0] for x in range(0, len(y_diff))]
    #correct = temp.count(0.0)
    #print("Number of correct predictions: ", correct)
    #print("Accuracy: {} % ".format(round((correct/len(temp))*100, 2)))

In [14]:
time_start = time.time()
gbrm = GradientBoostingRegressor(learning_rate=0.1, max_depth= 8, min_samples_leaf = 100, 
                                 min_samples_split = 10, 
                                  random_state = 23, subsample= 0.8) 
gbrm.fit(X_train, np.ravel(y_train))
print('time taken: ', time.time()-time_start, 's')
y_pred = np.array([[x] for x in gbrm.predict(X_test)])
display_results(y_pred, y_test)


time taken:  6.871137380599976 s
Predicted 1-5: [226882 230524 429566 139696 204170 226634 213542 189320 162491 200684]
Actual 1-5: [181000 240000 600000  40000 230000 330000 280000 189000 127000 271000]
Difference:  [  45882   -9476 -170434   99696  -25830 -103366  -66458     320   35491
  -70316]
RMSE:  272.32159507583384
R-squared:  0.34017585156212704


## Neural Network using Tensorflow

In [15]:
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Input, Activation, Dropout
from sklearn.metrics import mean_squared_error
from tensorflow.keras.losses import MeanSquaredLogarithmicError
from tensorflow.keras.optimizers import Adam


from math import sqrt

In [16]:
tf.__version__

'2.7.0'

In [17]:
input_layer = Input(shape=(X.shape[1],))
dense_layer_1 = Dense(100, activation='relu')(input_layer)
dense_layer_2 = Dense(200, activation='relu')(dense_layer_1)
dense_layer_3 = Dense(50, activation='relu')(dense_layer_2)
output = Dense(1)(dense_layer_3)

model = Model(inputs=input_layer, outputs=output)
model.compile(loss="mean_squared_error" , optimizer="adam", metrics=["mean_squared_error"])

In [18]:
history = model.fit(X_train, y_train, batch_size=50, epochs=50, verbose=1, validation_split=0.2)

Epoch 1/50
690/690 [==============================] - 1s 1ms/step - loss: 32689608704.0000 - mean_squared_error: 32689608704.0000 - val_loss: 19475380224.0000 - val_mean_squared_error: 19475380224.0000
Epoch 2/50
690/690 [==============================] - 1s 1ms/step - loss: 16080471040.0000 - mean_squared_error: 16080468992.0000 - val_loss: 18407174144.0000 - val_mean_squared_error: 18407174144.0000
Epoch 3/50
690/690 [==============================] - 1s 985us/step - loss: 15260067840.0000 - mean_squared_error: 15260067840.0000 - val_loss: 17745244160.0000 - val_mean_squared_error: 17745244160.0000
Epoch 4/50
690/690 [==============================] - 1s 1ms/step - loss: 14784906240.0000 - mean_squared_error: 14784906240.0000 - val_loss: 17352886272.0000 - val_mean_squared_error: 17352886272.0000
Epoch 5/50
690/690 [==============================] - 1s 1ms/step - loss: 14485051392.0000 - mean_squared_error: 14485051392.0000 - val_loss: 17088897024.0000 - val_mean_squared_error: 17088

690/690 [==============================] - 1s 1ms/step - loss: 13647648768.0000 - mean_squared_error: 13647648768.0000 - val_loss: 16221144064.0000 - val_mean_squared_error: 16221144064.0000
Epoch 42/50
690/690 [==============================] - 1s 996us/step - loss: 13633670144.0000 - mean_squared_error: 13633670144.0000 - val_loss: 16213843968.0000 - val_mean_squared_error: 16213843968.0000
Epoch 43/50
690/690 [==============================] - 1s 1ms/step - loss: 13625812992.0000 - mean_squared_error: 13625812992.0000 - val_loss: 16283538432.0000 - val_mean_squared_error: 16283538432.0000
Epoch 44/50
690/690 [==============================] - 1s 995us/step - loss: 13616848896.0000 - mean_squared_error: 13616848896.0000 - val_loss: 16274065408.0000 - val_mean_squared_error: 16274065408.0000
Epoch 45/50
690/690 [==============================] - 1s 1ms/step - loss: 13621510144.0000 - mean_squared_error: 13621510144.0000 - val_loss: 16230855680.0000 - val_mean_squared_error: 1623085568

In [19]:
print ("Model train metrics:")
pred_train = model.predict(X_train)
display_results(pred_train, y_train)

print("Model test metrics:")
pred_test = model.predict(X_test)
display_results(pred_test, y_test)

Model train metrics:
Predicted 1-5: [199785 340171 212267 303016 183237 421433 153186 356430 254625 306124]
Actual 1-5: [121000 400000 240000 415000  59000 340000 230000 363000 132000 184000]
Difference:  [  78785  -59829  -27733 -111984  124237   81433  -76814   -6570  122625
  122124]
RMSE:  276.4921303229691
R-squared:  0.29604125972386364
Model test metrics:
Predicted 1-5: [245437 228091 419986 145952 210530 247949 221970 185267 167657 186925]
Actual 1-5: [181000 240000 600000  40000 230000 330000 280000 189000 127000 271000]
Difference:  [  64437  -11909 -180014  105952  -19470  -82051  -58030   -3733   40657
  -84075]
RMSE:  274.7232877351471
R-squared:  0.3247491910530127


In [20]:
hidden_units1 = 160
hidden_units2 = 480
hidden_units3 = 256
learning_rate = 0.01

# build the model
model = Sequential([
    Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
    Dense(1, kernel_initializer='normal', activation='linear')
  ])

# loss function
msle = MeanSquaredLogarithmicError()
model.compile(
    loss=msle, 
    optimizer=Adam(learning_rate=learning_rate), 
    metrics=[msle]
)
# train the model
history = model.fit(
    X_train, 
    y_train, 
    epochs=50, 
    batch_size=64,
    validation_split=0.2
)

Epoch 1/50
539/539 [==============================] - 2s 2ms/step - loss: 1.8983 - mean_squared_logarithmic_error: 1.8971 - val_loss: 0.3096 - val_mean_squared_logarithmic_error: 0.3102
Epoch 2/50
539/539 [==============================] - 1s 2ms/step - loss: 0.3000 - mean_squared_logarithmic_error: 0.2999 - val_loss: 0.2867 - val_mean_squared_logarithmic_error: 0.2873
Epoch 3/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2860 - mean_squared_logarithmic_error: 0.2859 - val_loss: 0.2790 - val_mean_squared_logarithmic_error: 0.2796
Epoch 4/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2818 - mean_squared_logarithmic_error: 0.2819 - val_loss: 0.2762 - val_mean_squared_logarithmic_error: 0.2768
Epoch 5/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2809 - mean_squared_logarithmic_error: 0.2808 - val_loss: 0.2741 - val_mean_squared_logarithmic_error: 0.2746
Epoch 6/50
539/539 [==============================] - 1s 2ms/step - lo

Epoch 45/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2711 - mean_squared_logarithmic_error: 0.2711 - val_loss: 0.2706 - val_mean_squared_logarithmic_error: 0.2711
Epoch 46/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2715 - mean_squared_logarithmic_error: 0.2714 - val_loss: 0.2677 - val_mean_squared_logarithmic_error: 0.2682
Epoch 47/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2714 - mean_squared_logarithmic_error: 0.2713 - val_loss: 0.2678 - val_mean_squared_logarithmic_error: 0.2683
Epoch 48/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2715 - mean_squared_logarithmic_error: 0.2716 - val_loss: 0.2684 - val_mean_squared_logarithmic_error: 0.2689
Epoch 49/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2720 - mean_squared_logarithmic_error: 0.2720 - val_loss: 0.2775 - val_mean_squared_logarithmic_error: 0.2780
Epoch 50/50
539/539 [==============================] - 1s 2ms/ste

In [21]:
print ("Model train metrics:")
pred_train = model.predict(X_train)
display_results(pred_train, y_train)

print("Model test metrics:")
pred_test = model.predict(X_test)
display_results(pred_test, y_test)

Model train metrics:
Predicted 1-5: [190202 299964 160564 248229 154128 359592 135760 313821 226208 256825]
Actual 1-5: [121000 400000 240000 415000  59000 340000 230000 363000 132000 184000]
Difference:  [  69202 -100036  -79436 -166771   95128   19592  -94240  -49179   94208
   72825]
RMSE:  273.1191861605628
R-squared:  0.2684701184340258
Model test metrics:
Predicted 1-5: [221109 185898 356501 128576 188118 208981 212616 165051 144984 161581]
Actual 1-5: [181000 240000 600000  40000 230000 330000 280000 189000 127000 271000]
Difference:  [  40109  -54102 -243499   88576  -41882 -121019  -67384  -23949   17984
 -109419]
RMSE:  271.1757287587627
R-squared:  0.297153027501567


In [22]:
hidden_units1 = 512
hidden_units2 = 256
hidden_units3 = 128
hidden_units4 = 256
hidden_units5 = 128
learning_rate = 0.1

# build the model
model = Sequential([
    Dense(hidden_units1, kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
    Dense(hidden_units2, kernel_initializer='normal', activation='relu'),
   Dropout(0.2),
    Dense(hidden_units3, kernel_initializer='normal', activation='relu'),
    Dropout(0.2),
   # Dense(hidden_units4, kernel_initializer='normal', activation='relu'),
   # Dropout(0.2),
    #Dense(hidden_units5, kernel_initializer='normal', activation='relu'),
    Dense(1, kernel_initializer='normal', activation='linear')
  ])

# loss function
msle = MeanSquaredLogarithmicError()
model.compile(
    loss=msle, 
    optimizer=Adam(learning_rate=learning_rate), 
    metrics=[msle]
)
# train the model
history = model.fit(
    X_train, 
    y_train, 
    epochs=50, 
    batch_size=64,
    validation_split=0.2
)

Epoch 1/50
539/539 [==============================] - 2s 2ms/step - loss: 0.7849 - mean_squared_logarithmic_error: 0.7846 - val_loss: 0.2811 - val_mean_squared_logarithmic_error: 0.2816
Epoch 2/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2952 - mean_squared_logarithmic_error: 0.2952 - val_loss: 0.2801 - val_mean_squared_logarithmic_error: 0.2807
Epoch 3/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2954 - mean_squared_logarithmic_error: 0.2953 - val_loss: 0.2742 - val_mean_squared_logarithmic_error: 0.2748
Epoch 4/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2938 - mean_squared_logarithmic_error: 0.2937 - val_loss: 0.2735 - val_mean_squared_logarithmic_error: 0.2741
Epoch 5/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2944 - mean_squared_logarithmic_error: 0.2943 - val_loss: 0.2778 - val_mean_squared_logarithmic_error: 0.2785
Epoch 6/50
539/539 [==============================] - 1s 2ms/step - lo

Epoch 45/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2896 - mean_squared_logarithmic_error: 0.2896 - val_loss: 0.2849 - val_mean_squared_logarithmic_error: 0.2854
Epoch 46/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2898 - mean_squared_logarithmic_error: 0.2899 - val_loss: 0.2722 - val_mean_squared_logarithmic_error: 0.2727
Epoch 47/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2933 - mean_squared_logarithmic_error: 0.2932 - val_loss: 0.2821 - val_mean_squared_logarithmic_error: 0.2826
Epoch 48/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2886 - mean_squared_logarithmic_error: 0.2885 - val_loss: 0.2801 - val_mean_squared_logarithmic_error: 0.2807
Epoch 49/50
539/539 [==============================] - 1s 2ms/step - loss: 0.2917 - mean_squared_logarithmic_error: 0.2917 - val_loss: 0.2912 - val_mean_squared_logarithmic_error: 0.2916
Epoch 50/50
539/539 [==============================] - 1s 2ms/ste

In [23]:
print ("Model train metrics:")
pred_train = model.predict(X_train)
display_results(pred_train, y_train)

print("Model test metrics:")
pred_test = model.predict(X_test)
display_results(pred_test, y_test)

Model train metrics:
Predicted 1-5: [166604 265999 153658 204179 129449 364457 137466 250076 209123 182444]
Actual 1-5: [121000 400000 240000 415000  59000 340000 230000 363000 132000 184000]
Difference:  [  45604 -134001  -86342 -210821   70449   24457  -92534 -112924   77123
   -1556]
RMSE:  279.22313317120444
R-squared:  0.19301819793295927
Model test metrics:
Predicted 1-5: [184362 171441 348734 118553 156858 228950 198373 150297 137329 139569]
Actual 1-5: [181000 240000 600000  40000 230000 330000 280000 189000 127000 271000]
Difference:  [   3362  -68559 -251266   78553  -73142 -101050  -81627  -38703   10329
 -131431]
RMSE:  277.99211007415505
R-squared:  0.21156544704176483
